In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from newspaper import Article
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs
import datetime
import csv
import sys
import schedule
import os


def Crawling() :
    
    url='https://www.newsnow.co.uk/h/Industry+Sectors/Commodities/Energy/Natural+Gas?type=ln'

    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver_path = '/usr/local/bin/chromedriver'
    driver=webdriver.Chrome(driver_path, options=options)
    driver.implicitly_wait(3)
    driver.get(url)

    res = requests.get(url)
    soup = bs(res.text, 'lxml')
    div = soup.select_one("div.rs-grid.rs-grid--skeleton.js-skeleton > main > div.rs-newsbox.js-newsbox.js-newsmain.js-central_ln_wrap > div").findAll('div', {"class" : "hl"})


    temp_link_list = []
    link_list = []
    name_list = []
    article_list = []
    date_list = []

    
    for hl in div :
        data = hl.find('a', {"class" : "hll"})
        time = hl.find('span', {'class' : "time"})
        
        date = time.text
        date = pd.to_datetime(date)
        date = date.strftime("%Y-%m-%d %H:%M:%S")
        date_list.append(date)

        temp_link_list.append(data.get('href'))
        name_list.append(data.text)


    for url in temp_link_list :
        res = requests.get(url)
        soup = bs(res.text, 'lxml')
        url = soup.select_one('script').get_text().split(" ")[13]
        urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', str(url))
        urls = ''.join(urls)
        link_list.append(urls)


    for url in link_list :
        article = Article(url)
        try :
            article.download()
            article.parse()
            if len(article.text) > 30000 :
                article_list.append(article.summary)
            else :
                article_list.append(article.text)
        except :
            article_list.append('')


    df = pd.DataFrame({'link' : link_list,
                       'title': name_list,
                       'time': date_list,
                       'article': article_list
                      })
    

    return df


In [2]:
df = Crawling()

/var/folders/7n/qbcvswc55hzd4lnphhs0grb40000gn/T/ipykernel_66675/839339383.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(driver_path, options=options)


In [3]:
df.sort_index(axis=0, ascending=False, ignore_index=True, inplace=True)

In [4]:
df['link'] = df['link'].astype('string')
df['title'] = df['title'].astype('string')
df['time'] = df['time'].astype('string')
df['article'] = df['article'].astype('string')

In [5]:
df

,link,title,time,article
0,https://www.businesswire.com/news/home/2022080...,NOG Announces Second Quarter 2022 Results,2022-08-03 21:24:00,MINNEAPOLIS--(BUSINESS WIRE)--Northern Oil and...
1,https://www.naturalgasintel.com/chesapeake-ene...,Chesapeake Energy Exiting Eagle Ford Shale in ...,2022-08-03 21:29:00,Chesapeake Energy Corp. will exit the Eagle Fo...
2,https://www.reuters.com/markets/commodities/le...,"Less U.S. gas to Asia, Freeport explosion redu...",2022-08-03 21:39:00,The first trial run with a Post-Panamax cargo ...
3,https://www.bnnbloomberg.ca/natural-gas-surges...,Natural Gas Surges With US LNG Export Terminal...,2022-08-03 21:41:00,(Bloomberg) -- US natural gas prices surged af...
4,https://www.investing.com/news/commodities-new...,U.S. regulator to allow Freeport LNG to resume...,2022-08-03 21:41:00,© Reuters. By Liz Hampton (Reuters) - The se...
...,...,...,...,...
110,https://asia.nikkei.com/Business/Energy/Japan-...,"Japan tells Mitsui, Mitsubishi to move 'positi...",2022-08-05 07:26:00,TOKYO (Reuters) -- Japan's government has aske...
111,https://economictimes.indiatimes.com/industry/...,Piped cooking gas price hiked by Rs 2.63 per u...,2022-08-05 07:35:00,The price of cooking gas piped to household ki...
112,https://www.offshore-energy.biz/eia-us-weekly-...,EIA: US weekly LNG exports up by four LNG tankers,2022-08-05 07:43:00,"August 5, 2022, by Sanja Pekic According to t..."
113,https://www.bnnbloomberg.ca/european-gas-set-f...,European Gas Set for Another Weekly Gain on Ru...,2022-08-05 08:17:00,(Bloomberg) -- European natural gas prices hea...


In [6]:
df.to_csv('news_data_natural_gas.csv',index=False)